 
# Alpha and Beta Diversity

Different higher-level measures are often used to describe the microbiome in a sample. These do not provide information on changes in the abundance of specific taxa but allow us to access a broader change or difference in the composition of microorganisms. Alpha and beta diversity are examples of such measures.

Different measures exist to estimate diversity within a single sample, jointly called alpha diversity. The different measures reflect the richness (number) or distribution (evenness) of a microbial sample or aim to reflect a combination of both properties.

Rarefaction curves are often used when calculating alpha diversity indices because increasing numbers of sequenced taxa allow increasingly accurate estimates of total population diversity. Rarefaction curves can therefore be used to estimate the full sample richness, as compared to the observed sample richness.

While alpha diversity is a measure of microbiome diversity applicable to a single sample, beta diversity is a measure of the similarity or dissimilarity of two communities. As for alpha diversity, many indices exist, each reflecting different aspects of community heterogeneity. Key differences relate to how the indices value variation in rare species if they consider presence/absence only or incorporate abundance, and how they interpret shared absence. Bray-Curtis dissimilarity is a popular measure that considers both size (overall abundance per sample) and shape (abundance of each taxon) of the communities (Bray, 1957). Beta diversity is an essential measure for many popular statistical methods in ecology, such as ordination-based methods, and is widely used for studying the association between environmental variables and microbial composition.

In summary, alpha diversity measures can be seen as a summary statistic of a single population (within-sample diversity), while beta diversity measures are estimates of similarity or dissimilarity between populations (between samples).

**Source**: (https://biomcare.com/info/key-terms-in-microbiome-projects/)

### STEP : Diversity Analysis

Using QIIME2 to create diversity analisys graphs and calculations.

- [QIIME2 Workflow Overview](https://docs.qiime2.org/2022.8/tutorials/overview/)


#### Methods
- [diversity](https://docs.qiime2.org/2022.8/plugins/available/diversity/)
- [diversity alpha](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha/)
- [diversity alpha_phylogenetic](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-phylogenetic/)
- [diversity beta](https://docs.qiime2.org/2022.8/plugins/available/diversity/beta/)
- [diversity core_metrics](https://docs.qiime2.org/2022.8/plugins/available/diversity/core-metrics/)
- [diversity alpha_group_significance](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-group-significance/)
- [diversity beta_group_significance](https://docs.qiime2.org/2022.8/plugins/available/diversity/beta-group-significance/)
- [feature_table core_features](https://docs.qiime2.org/2022.8/plugins/available/feature-table/core-features/)
- [feature_table summarize](https://docs.qiime2.org/2022.8/plugins/available/feature-table/summarize/)
- [taxa filter-table](https://docs.qiime2.org/2022.8/plugins/available/taxa/filter-table/)
- [taxa collapse](https://docs.qiime2.org/2022.8/plugins/available/taxa/collapse/)

## Setup and settings

In [1]:
# Importing packages
import os
import pandas as pd
from qiime2 import Artifact
from qiime2 import Visualization
from qiime2 import Metadata

from qiime2.plugins.phylogeny.pipelines import align_to_tree_mafft_fasttree

from qiime2.plugins.diversity.pipelines import alpha
from qiime2.plugins.diversity.pipelines import beta
from qiime2.plugins.diversity.pipelines import core_metrics
from qiime2.plugins.diversity.pipelines import alpha_phylogenetic

from qiime2.plugins.diversity.visualizers import alpha_group_significance
from qiime2.plugins.diversity.visualizers import beta_group_significance
from qiime2.plugins.diversity.visualizers import alpha_correlation
from qiime2.plugins.diversity.visualizers import beta_rarefaction

from qiime2.plugins.taxa.methods import filter_table
from qiime2.plugins.taxa.methods import collapse

from qiime2.plugins.feature_table.visualizers import tabulate_seqs
from qiime2.plugins.feature_table.visualizers import summarize
from qiime2.plugins.feature_table.visualizers import core_features
from qiime2.plugins.diversity.pipelines import core_metrics_phylogenetic

from qiime2.plugins.feature_table.methods import filter_samples
from qiime2.plugins.feature_table.methods import filter_seqs

from qiime2.plugins.alignment.methods import mafft


import matplotlib.pyplot as plt

%matplotlib inline

### Receiving the parameters

The following cell can receive parameters using the [papermill](https://papermill.readthedocs.io/en/latest/) tool.

In [2]:
base_dir = os.path.join('/', 'home')
metadata_file = os.path.abspath(os.path.join(base_dir, 'data', 'metadatada.tsv'))
experiment_name = ''
class_col = ''
replace_files = False

In [3]:
# Parameters
experiment_name = "thayane-PM-joined-trim"
base_dir = "/home/lauro/nupeb/rede-micro/redemicro-thayane"
manifest_file = (
    "/home/lauro/nupeb/rede-micro/redemicro-thayane/data/manifest-single.csv"
)
metadata_file = "/home/lauro/nupeb/rede-micro/redemicro-thayane/data/metadata.tsv"
class_col = "above_10"
classifier_file = (
    "/home/lauro/nupeb/16S_classifiers_qiime2/silva-138-99-nb-classifier.qza"
)
replace_files = False
phred = 20
trunc_f = 0
trunc_r = 0
overlap = 12
threads = 6


In [4]:
experiment_folder = os.path.abspath(os.path.join(base_dir, 'experiments', experiment_name))
img_folder = os.path.abspath(os.path.join(experiment_folder, 'imgs'))

### Defining names, paths and flags

In [5]:
# QIIME2 Artifacts folder
qiime_folder = os.path.join(experiment_folder, 'qiime-artifacts')

# Input - DADA2 Artifacts
dada2_tabs_path = os.path.join(qiime_folder, 'dada2-tabs.qza')
dada2_reps_path = os.path.join(qiime_folder, 'dada2-reps.qza')
dada2_stat_path = os.path.join(qiime_folder, 'dada2-stat.qza')

# Input - Taxonaomic Artifacts
taxonomy_path = os.path.join(qiime_folder, 'metatax.qza')

# Create folder to store Alpha files
alpha_path = os.path.join(qiime_folder, 'alpha-analysis')
if not os.path.exists(alpha_path):
    os.makedirs(alpha_path)
    print(f'The new directory is created in {alpha_path}')
    
# Create folder to store Beta files
beta_path = os.path.join(qiime_folder, 'beta-analysis')
if not os.path.exists(beta_path):
    os.makedirs(beta_path)
    print(f'The new directory is created in {beta_path}')

# Output -Diversity Artifacts
alpha_diversity_path = os.path.join(alpha_path, 'alpha-diversity.qza')
alpha_diversity_view_path = os.path.join(alpha_path, 'alpha-diversity.qzv')
beta_diversity_path = os.path.join(beta_path, 'beta-diversity.qza')
beta_diversity_view_path = os.path.join(beta_path, 'beta-diversity.qzv')

In [6]:
def filter_and_collapse(tab, seqs, tax, meta, lvl, exclude=True, exclude_list='uncultured,unidentified,metagenome'):
    from qiime2.plugins.taxa.methods import collapse
    from qiime2.plugins.taxa.methods import filter_table
    from qiime2.plugins.feature_table.methods import filter_seqs
    from qiime2.plugins.feature_table.visualizers import summarize
    
    to_include = ('d', 'p', 'c', 'o', 'f', 'g', 's')[lvl-1]
    to_include += '__'
    to_exclude = exclude_list if exclude else None
    
    filtered_tabs = filter_table(
        table=tab, 
        taxonomy=tax,
        include=to_include,
        exclude=to_exclude,
        mode='contains').filtered_table
    
    filtered_seqs = filter_seqs(
        data = seqs,
        table = filtered_tabs,
    ).filtered_data
    
    collapsed_table = collapse(table=filtered_tabs, taxonomy=tax, level=lvl).collapsed_table
    collapsed_table_view = summarize(table=collapsed_table, sample_metadata=meta).visualization
    
    return collapsed_table, collapsed_table_view, filtered_seqs

## Step execution

### Load input files

This Step import the QIIME2 `FeatureTable[Frequency]` Artifact and the `Metadata` file.

In [7]:
#Load Metadata
metadata_qa = Metadata.load(metadata_file)

#Load FeatureTable[Frequency]
tabs = Artifact.load(dada2_tabs_path)
tabs_df = tabs.view(Metadata).to_dataframe().T

# FeatureData[Sequence]
reps = Artifact.load(dada2_reps_path)

# FeatureData[Taxonomy]
tax = Artifact.load(taxonomy_path)

In [8]:
# Filter FeatureTable[Frequency | RelativeFrequency | PresenceAbsence | Composition] based on Metadata sample ID values
tabs = filter_samples(
    table=tabs,
    metadata=metadata_qa,
).filtered_table
# Filter SampleData[SequencesWithQuality | PairedEndSequencesWithQuality | JoinedSequencesWithQuality] based on Metadata sample ID values; returns FeatureData[Sequence | AlignedSequence]
reps = filter_seqs(
    data=reps,
    table=tabs,
).filtered_data

{'min_frequency': 0, 'max_frequency': None, 'min_features': 0, 'max_features': None, 'metadata': Metadata
--------
44 IDs x 8 columns
class-straw:        ColumnProperties(type='categorical')
local-coleta:       ColumnProperties(type='categorical')
idade:              ColumnProperties(type='numeric')
idade-menarca:      ColumnProperties(type='numeric')
tempo-menopausa:    ColumnProperties(type='numeric')
above_10:           ColumnProperties(type='categorical')
menopausa-age-qcut: ColumnProperties(type='categorical')
menopausa-age-bins: ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'where': None, 'exclude_ids': False, 'filter_empty_features': True, 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
{'min_frequency': 0, 'max_frequency': None, 'min_features': 0, 'max_features': None, 'metadata': Metadata
--------
44 IDs x 8 columns
class-straw:        ColumnProperties(type='categorical')
local-coleta:       ColumnP

{'metadata': None, 'where': None, 'exclude_ids': False, 'data': 00467836d98f063b17a325cd51d97330    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)), ((A...
0053504a75cbad644658d68e0d32aa7e    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)), ((A...
0060fd12eeb7e0c9151ed9bd939ec2ab    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)), ((A...
00aa928d554b4d273a8c64786d4912dd    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)), ((A...
00b396f20b86b3eb437d46d5d41ecca4    (((T)), ((T)), ((A)), ((G)), ((G)), ((A)), ((A...
                                                          ...                        
ffb2262b8183655d421ec92b0c76fb7f    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)), ((A...
ffcebd381e84cfe83cdacd5f6a6c25b4    (((G)), ((C)), ((G)), ((C)), ((G)), ((A)), ((A...
ffd728469d288c1ecdb32ff2bbc18598    (((T)), ((G)), ((A)), ((G)), ((G)), ((A)), ((A...
ffeae2fdd8c2c7b1d1c1c1d7a33944c8    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)), ((A...
fff0ea2bfed7ded0d6e2d0b6db49329e    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)

{'metadata': None, 'where': None, 'exclude_ids': False, 'data': 00467836d98f063b17a325cd51d97330    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)), ((A...
0053504a75cbad644658d68e0d32aa7e    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)), ((A...
0060fd12eeb7e0c9151ed9bd939ec2ab    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)), ((A...
00aa928d554b4d273a8c64786d4912dd    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)), ((A...
00b396f20b86b3eb437d46d5d41ecca4    (((T)), ((T)), ((A)), ((G)), ((G)), ((A)), ((A...
                                                          ...                        
ffb2262b8183655d421ec92b0c76fb7f    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)), ((A...
ffcebd381e84cfe83cdacd5f6a6c25b4    (((G)), ((C)), ((G)), ((C)), ((G)), ((A)), ((A...
ffd728469d288c1ecdb32ff2bbc18598    (((T)), ((G)), ((A)), ((G)), ((G)), ((A)), ((A...
ffeae2fdd8c2c7b1d1c1c1d7a33944c8    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)), ((A...
fff0ea2bfed7ded0d6e2d0b6db49329e    (((T)), ((G)), ((G)), ((G)), ((G)), ((A)

## Alpha diversity analysis

#### Reference
- [The Use and Types of Alpha-Diversity Metrics in Microbial NGS](https://www.cd-genomics.com/microbioseq/the-use-and-types-of-alpha-diversity-metrics-in-microbial-ngs.html)
- [Alpha diversity metrics](http://scikit-bio.org/docs/0.2.0/generated/skbio.diversity.alpha.html)

#### Methods
- [diversity alpha](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha/): Computes a user-specified alpha diversity metric for all samples in a
feature table.
- [diversity alpha_phylogenetic](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-phylogenetic/): Computes a user-specified phylogenetic alpha diversity metric for all
samples in a feature table.
- [diversity alpha_correlation](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-correlation/): Determine whether numeric sample metadata columns are correlated with alpha diversity.
- [diversity alpha_group_significance](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-group-significance/): Visually and statistically compare groups of alpha diversity values.

### Compute Alpha Diversity vectors
- [diversity alpha](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha/): Computes a user-specified alpha diversity metric for all samples in a feature table.
- [Alpha diversity metrics](http://scikit-bio.org/docs/0.2.0/generated/skbio.diversity.alpha.html)
 - Choices: ('ace', 'berger_parker_d', 'brillouin_d', 'chao1', 'chao1_ci', 'dominance', 'doubles', 'enspie', 'esty_ci', 'fisher_alpha', 'gini_index', 'goods_coverage', 'heip_e', 'kempton_taylor_q', 'lladser_pe', 'margalef', 'mcintosh_d', 'mcintosh_e', 'menhinick', 'michaelis_menten_fit', 'observed_features', 'osd', 'pielou_e', 'robbins', 'shannon', 'simpson', 'simpson_e', 'singles', 'strong')

In [9]:
metrics = ('ace', 'berger_parker_d', 'brillouin_d', 'chao1', 'chao1_ci', 'dominance', 'doubles', 'enspie', 'esty_ci', 'fisher_alpha', 'gini_index', 'goods_coverage', 'heip_e', 'kempton_taylor_q', 'lladser_pe', 'margalef', 'mcintosh_d', 'mcintosh_e', 'menhinick', 'michaelis_menten_fit', 'observed_features', 'osd', 'pielou_e', 'robbins', 'shannon', 'simpson', 'simpson_e', 'singles', 'strong')

# Sugestão de valores para Alpha diversity
# chao1 e observed_features (riqueza); shannon e simpson (diversidade - que levam em consideração riqueza e equitabilidade).
metrics = ('chao1', 'chao1_ci', 'observed_features', 'shannon', 'simpson', 'simpson_e')
alpha_diversities = dict()
for metric in metrics:
    print(f"Calculating alpha diversity: {metric}")
    try:
        alpha_diversity = alpha(table=tabs, metric=metric).alpha_diversity
        alpha_diversities[metric] = alpha_diversity
        # Save SampleData[AlphaDiversity] Artifact
        file_path = os.path.join(alpha_path, f'alpha-values-{metric}.qza')
        alpha_diversity.save(file_path)
        print(f"DONE: Calculating alpha diversity: {metric}")
    except Exception as e:
        print(f"ERROR: Calculating alpha diversity: {metric}")
        print(e)

Calculating alpha diversity: chao1
{'metric': 'chao1', 'table': <artifact: FeatureTable[Frequency] uuid: b8c70aab-e476-498b-9b85-38d174c191e2>}
{'metric': 'chao1', 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
{'metric': 'chao1', 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
DONE: Calculating alpha diversity: chao1
Calculating alpha diversity: chao1_ci
{'metric': 'chao1_ci', 'table': <artifact: FeatureTable[Frequency] uuid: b8c70aab-e476-498b-9b85-38d174c191e2>}
{'metric': 'chao1_ci', 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
{'metric': 'chao1_ci', 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
DONE: Calculating alpha diversity: chao1_ci
Calculating alpha diversity: observed_features
{'metric': 'observed_features', 'table': <artifact: FeatureTable[Frequency] uuid: b8c70aab-e476-498b-9b85-38d174c191e2>}
{'table': 2403 x 44 <class 'bio

DONE: Calculating alpha diversity: observed_features
Calculating alpha diversity: shannon
{'metric': 'shannon', 'table': <artifact: FeatureTable[Frequency] uuid: b8c70aab-e476-498b-9b85-38d174c191e2>}
{'drop_undefined_samples': False, 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
{'drop_undefined_samples': False, 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
DONE: Calculating alpha diversity: shannon
Calculating alpha diversity: simpson
{'metric': 'simpson', 'table': <artifact: FeatureTable[Frequency] uuid: b8c70aab-e476-498b-9b85-38d174c191e2>}
{'metric': 'simpson', 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
{'metric': 'simpson', 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
DONE: Calculating alpha diversity: simpson
Calculating alpha diversity: simpson_e
{'metric': 'simpson_e', 'table': <artifact: FeatureTable[Frequency] uuid: b8c7

{'metric': 'simpson_e', 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
{'metric': 'simpson_e', 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
DONE: Calculating alpha diversity: simpson_e


### Create Phylogenetic inference

- [alignment align_to_tree_mafft_fasttree](https://docs.qiime2.org/2022.8/plugins/available/phylogeny/align-to-tree-mafft-fasttree/): Build a phylogenetic tree using fasttree and mafft alignment

This pipeline will start by creating a sequence alignment using MAFFT,
after which any alignment columns that are phylogenetically uninformative
or ambiguously aligned will be removed (masked). The resulting masked
alignment will be used to infer a phylogenetic tree and then subsequently
rooted at its midpoint. Output files from each step of the pipeline will be
saved. This includes both the unmasked and masked MAFFT alignment from
q2-alignment methods, and both the rooted and unrooted phylogenies from
q2-phylogeny methods.


Returns
- alignment : FeatureData[AlignedSequence] : The aligned sequences.
- masked_alignment : FeatureData[AlignedSequence] : The masked alignment.
- tree : Phylogeny[Unrooted] : The unrooted phylogenetic tree.
- rooted_tree : Phylogeny[Rooted] : The rooted phylogenetic tree.

In [10]:
mafft_alignment, mafft_masked_alignment, mafft_tree, mafft_rooted_tree = align_to_tree_mafft_fasttree(
    sequences=reps, n_threads=6, )

{'n_threads': 6, 'mask_max_gap_frequency': 1.0, 'mask_min_conservation': 0.4, 'parttree': False, 'sequences': <artifact: FeatureData[Sequence] uuid: 3f3325da-33f0-487c-a467-f2a98a7de75c>}
{'n_threads': 6, 'parttree': False, 'sequences': <q2_types.feature_data._format.DNAFASTAFormat object at 0x7f634c4ac2b0>}
{'n_threads': 6, 'parttree': False, 'sequences': <q2_types.feature_data._format.DNAFASTAFormat object at 0x7f634c4ac2b0>}
Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 6 /tmp/qiime2-archive-9psyjbp5/3f3325da-33f0-487c-a467-f2a98a7de75c/data/dna-sequences.fasta



inputfile = orig
2403 x 467 - 24 d
nthread = 6
nthreadpair = 6
nthreadtb = 6
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
  101 / 2403 (thread    2)

  401 / 2403 (thread    1)

  701 / 2403 (thread    4)

 1201 / 2403 (thread    2)

...

     48.11 seconds: ML split tests for   2000 of   2302 internal splits


     48.41 seconds: ML split tests for   2100 of   2302 internal splits


     48.71 seconds: ML split tests for   2200 of   2302 internal splits


     49.02 seconds: ML split tests for   2300 of   2302 internal splits
Total time: 49.04 seconds Unique: 2305/2403 Bad splits: 2/2302 Worst delta-LogLk 0.224


{'tree': <TreeNode, name: unnamed, internal node count: 2389, tips count: 2403>}
{'tree': <TreeNode, name: unnamed, internal node count: 2389, tips count: 2403>}


### Compute Alpha Diversity (Phylogeny)
- [diversity alpha_phylogenetic](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-phylogenetic/): Computes a user-specified phylogenetic alpha diversity metric for all samples in a feature table.
- Metrics: Choices ('faith_pd')

In [11]:
metrics = ('faith_pd', )
alpha_diversities_phylogenetic = dict()
for metric in metrics:
    print(f"Calculating alpha diversity: {metric}")
    try:
        alpha_diversity = alpha_phylogenetic(table=tabs, phylogeny=mafft_rooted_tree, metric=metric).alpha_diversity
        alpha_diversities_phylogenetic[metric] = alpha_diversity
        # Save Artifact
        file_path = os.path.join(alpha_path, f'alpha-phylogeny-{metric}.qza')
        alpha_diversity.save(file_path)
        print(f"DONE: Calculating alpha phylogeny: {metric}")
    except Exception as e:
        print(f"ERROR: Calculating alpha phylogeny: {metric}")

Calculating alpha diversity: faith_pd
{'metric': 'faith_pd', 'table': <artifact: FeatureTable[Frequency] uuid: b8c70aab-e476-498b-9b85-38d174c191e2>, 'phylogeny': <artifact: Phylogeny[Rooted] uuid: 56fe8435-a50d-44ba-adc0-e28c550ba0a2>}
{'table': <q2_types.feature_table._format.BIOMV210Format object at 0x7f6337a980d0>, 'phylogeny': <q2_types.tree._format.NewickFormat object at 0x7f6337a98640>}
{'table': <q2_types.feature_table._format.BIOMV210Format object at 0x7f6337a980d0>, 'phylogeny': <q2_types.tree._format.NewickFormat object at 0x7f6337a98640>}


DONE: Calculating alpha phylogeny: faith_pd


- [core-metrics-phylogenetic](https://docs.qiime2.org/2023.7/plugins/available/diversity/core-metrics-phylogenetic/)

In [12]:
s_depth = int(tabs.view(pd.DataFrame).sum(axis=1).min())
results = core_metrics_phylogenetic(
    table = tabs,
    phylogeny = mafft_rooted_tree,
    sampling_depth = s_depth,
    metadata = metadata_qa,
    n_jobs_or_threads = 6,
)

{'sampling_depth': 564, 'metadata': Metadata
--------
44 IDs x 8 columns
class-straw:        ColumnProperties(type='categorical')
local-coleta:       ColumnProperties(type='categorical')
idade:              ColumnProperties(type='numeric')
idade-menarca:      ColumnProperties(type='numeric')
tempo-menopausa:    ColumnProperties(type='numeric')
above_10:           ColumnProperties(type='categorical')
menopausa-age-qcut: ColumnProperties(type='categorical')
menopausa-age-bins: ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'n_jobs_or_threads': 6, 'table': <artifact: FeatureTable[Frequency] uuid: b8c70aab-e476-498b-9b85-38d174c191e2>, 'phylogeny': <artifact: Phylogeny[Rooted] uuid: 56fe8435-a50d-44ba-adc0-e28c550ba0a2>}
{'sampling_depth': 564, 'metadata': Metadata
--------
44 IDs x 8 columns
class-straw:        ColumnProperties(type='categorical')
local-coleta:       ColumnProperties(type='categorical')
idade:              ColumnProperties(type='n

{'drop_undefined_samples': False, 'table': 1252 x 44 <class 'biom.table.Table'> with 3591 nonzero entries (6% dense)}
{'drop_undefined_samples': False, 'table': 1252 x 44 <class 'biom.table.Table'> with 3591 nonzero entries (6% dense)}
{'drop_undefined_samples': False, 'table': 1252 x 44 <class 'biom.table.Table'> with 3591 nonzero entries (6% dense)}
{'drop_undefined_samples': False, 'table': 1252 x 44 <class 'biom.table.Table'> with 3591 nonzero entries (6% dense)}
{'n_jobs': 6, 'table': 1252 x 44 <class 'biom.table.Table'> with 3591 nonzero entries (6% dense)}
{'n_jobs': 6, 'table': 1252 x 44 <class 'biom.table.Table'> with 3591 nonzero entries (6% dense)}
{'n_jobs': 6, 'table': 1252 x 44 <class 'biom.table.Table'> with 3591 nonzero entries (6% dense)}
{'n_jobs': 6, 'table': 1252 x 44 <class 'biom.table.Table'> with 3591 nonzero entries (6% dense)}


/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1761: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


{'number_of_dimensions': None, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f63379eed60>}
{'number_of_dimensions': None, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f63379eed60>}
{'number_of_dimensions': None, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f6337a9e370>}
{'number_of_dimensions': None, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f6337a9e370>}
{'metadata': Metadata
--------
44 IDs x 8 columns
class-straw:        ColumnProperties(type='categorical')
local-coleta:       ColumnProperties(type='categorical')
idade:              ColumnProperties(type='numeric')
idade-menarca:      ColumnProperties(type='numeric')
tempo-menopausa:    ColumnProperties(type='numeric')
above_10:           ColumnProperties(type='categorical')
menopausa-age-qcut: ColumnProperties(type='categorical')
menopausa-age-bins: ColumnProperties(type='categorical')

Call to_dataframe() for a

{'metadata': Metadata
--------
44 IDs x 8 columns
class-straw:        ColumnProperties(type='categorical')
local-coleta:       ColumnProperties(type='categorical')
idade:              ColumnProperties(type='numeric')
idade-menarca:      ColumnProperties(type='numeric')
tempo-menopausa:    ColumnProperties(type='numeric')
above_10:           ColumnProperties(type='categorical')
menopausa-age-qcut: ColumnProperties(type='categorical')
menopausa-age-bins: ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'custom_axes': None, 'ignore_missing_samples': False, 'ignore_pcoa_features': False, 'pcoa': <skbio.stats.ordination._ordination_results.OrdinationResults object at 0x7f6337a1c760>}


{'table': <q2_types.feature_table._format.BIOMV210Format object at 0x7f6337a9eee0>, 'phylogeny': <q2_types.tree._format.NewickFormat object at 0x7f6337a82910>}
{'table': <q2_types.feature_table._format.BIOMV210Format object at 0x7f6337a9eee0>, 'phylogeny': <q2_types.tree._format.NewickFormat object at 0x7f6337a82910>}
{'threads': 6, 'bypass_tips': False, 'table': <q2_types.feature_table._format.BIOMV210Format object at 0x7f6337a1c550>, 'phylogeny': <q2_types.tree._format.NewickFormat object at 0x7f6337a1c6d0>}
{'threads': 6, 'bypass_tips': False, 'table': <q2_types.feature_table._format.BIOMV210Format object at 0x7f6337a1c550>, 'phylogeny': <q2_types.tree._format.NewickFormat object at 0x7f6337a1c6d0>}


{'threads': 6, 'bypass_tips': False, 'table': <q2_types.feature_table._format.BIOMV210Format object at 0x7f633786d3a0>, 'phylogeny': <q2_types.tree._format.NewickFormat object at 0x7f6337f02970>}
{'threads': 6, 'bypass_tips': False, 'table': <q2_types.feature_table._format.BIOMV210Format object at 0x7f633786d3a0>, 'phylogeny': <q2_types.tree._format.NewickFormat object at 0x7f6337f02970>}
{'number_of_dimensions': None, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f6337a9ef40>}
{'number_of_dimensions': None, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f6337a9ef40>}
{'number_of_dimensions': None, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f633786d190>}
{'number_of_dimensions': None, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f633786d190>}


/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.029575607816941275 and the largest is 1.220413148650924.
  warn(


{'metadata': Metadata
--------
44 IDs x 8 columns
class-straw:        ColumnProperties(type='categorical')
local-coleta:       ColumnProperties(type='categorical')
idade:              ColumnProperties(type='numeric')
idade-menarca:      ColumnProperties(type='numeric')
tempo-menopausa:    ColumnProperties(type='numeric')
above_10:           ColumnProperties(type='categorical')
menopausa-age-qcut: ColumnProperties(type='categorical')
menopausa-age-bins: ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'custom_axes': None, 'ignore_missing_samples': False, 'ignore_pcoa_features': False, 'pcoa': <skbio.stats.ordination._ordination_results.OrdinationResults object at 0x7f63379ee490>}
{'metadata': Metadata
--------
44 IDs x 8 columns
class-straw:        ColumnProperties(type='categorical')
local-coleta:       ColumnProperties(type='categorical')
idade:              ColumnProperties(type='numeric')
idade-menarca:      ColumnProperties(type='numeric')
te

In [13]:
results_info = [("rarefied_table", "FeatureTable[Frequency]", "The resulting rarefied feature table."),
("faith_pd_vector", "SampleData[AlphaDiversity]", "Vector of Faith PD values by sample."),
("observed_features_vector", "SampleData[AlphaDiversity]", "Vector of Observed Features values by sample."),
("shannon_vector", "SampleData[AlphaDiversity]", "Vector of Shannon diversity values by sample."),
("evenness_vector", "SampleData[AlphaDiversity]", "Vector of Pielou's evenness values by sample."),
("unweighted_unifrac_distance_matrix", "DistanceMatrix", "Matrix of unweighted UniFrac distances between pairs of samples."),
("weighted_unifrac_distance_matrix", "DistanceMatrix", "Matrix of weighted UniFrac distances between pairs of samples."),
("jaccard_distance_matrix", "DistanceMatrix", "Matrix of Jaccard distances between pairs of samples."),
("bray_curtis_distance_matrix", "DistanceMatrix", "Matrix of Bray-Curtis distances between pairs of samples."),
("unweighted_unifrac_pcoa_results", "PCoAResults", "PCoA matrix computed from unweighted UniFrac distances between samples."),
("weighted_unifrac_pcoa_results", "PCoAResults", "PCoA matrix computed from weighted UniFrac distances between samples."),
("jaccard_pcoa_results", "PCoAResults", "PCoA matrix computed from Jaccard distances between samples."),
("bray_curtis_pcoa_results", "PCoAResults", "PCoA matrix computed from Bray-Curtis distances between samples."),
("unweighted_unifrac_emperor", "Visualization", "Emperor plot of the PCoA matrix computed from unweighted UniFrac."),
("weighted_unifrac_emperor", "Visualization", "Emperor plot of the PCoA matrix computed from weighted UniFrac."),
("jaccard_emperor", "Visualization", "Emperor plot of the PCoA matrix computed from Jaccard."),
("bray_curtis_emperor", "Visualization", "Emperor plot of the PCoA matrix computed from Bray-Curtis.")]

In [14]:
for i, info in enumerate(results_info):
    r_id, r_type, r_desc = info
    #print(i, r_id, r_type)
    file_name = f"{r_id}.qzv"
    if r_type == "FeatureTable[Frequency]":
        pass
    elif r_id.endswith('emperor'):
        print(i, r_id, r_type)
        print(f"--- {r_desc} ---")
        file_name = os.path.join(beta_path, file_name)
        print(f'Saving emperor file at: {file_name}\n')
        results[i].save(filepath=file_name)

13 unweighted_unifrac_emperor Visualization
--- Emperor plot of the PCoA matrix computed from unweighted UniFrac. ---
Saving emperor file at: /home/lauro/nupeb/rede-micro/redemicro-thayane/experiments/thayane-PM-joined-trim/qiime-artifacts/beta-analysis/unweighted_unifrac_emperor.qzv

14 weighted_unifrac_emperor Visualization
--- Emperor plot of the PCoA matrix computed from weighted UniFrac. ---
Saving emperor file at: /home/lauro/nupeb/rede-micro/redemicro-thayane/experiments/thayane-PM-joined-trim/qiime-artifacts/beta-analysis/weighted_unifrac_emperor.qzv



15 jaccard_emperor Visualization
--- Emperor plot of the PCoA matrix computed from Jaccard. ---
Saving emperor file at: /home/lauro/nupeb/rede-micro/redemicro-thayane/experiments/thayane-PM-joined-trim/qiime-artifacts/beta-analysis/jaccard_emperor.qzv

16 bray_curtis_emperor Visualization
--- Emperor plot of the PCoA matrix computed from Bray-Curtis. ---
Saving emperor file at: /home/lauro/nupeb/rede-micro/redemicro-thayane/experiments/thayane-PM-joined-trim/qiime-artifacts/beta-analysis/bray_curtis_emperor.qzv



### Alpha diversity correlation

This method only process `numeric` columns.


In [15]:
methods = ('spearman', 'pearson')
numerics_cols = metadata_qa.filter_columns(column_type='numeric')
if numerics_cols.column_count > 0:
    for metric, alpha_values in alpha_diversities.items():
        for method in methods:
            try:
                corr_view = alpha_correlation(alpha_diversity=alpha_values, metadata=numerics_cols, 
                                          method=method, intersect_ids=True).visualization
                view_path = os.path.join(alpha_path, f'alpha-correlation-{metric}-{method}.qzv')
                corr_view.save(view_path)
                corr_view
                print(f"DONE: Calculating alpha correlation: {metric} {method}")
            except Exception as e:
                print(f"ERROR: Calculating alpha correlation: {metric} {method}")

{'metadata': Metadata
--------
44 IDs x 3 columns
idade:           ColumnProperties(type='numeric')
idade-menarca:   ColumnProperties(type='numeric')
tempo-menopausa: ColumnProperties(type='numeric')

Call to_dataframe() for a tabular representation., 'method': 'spearman', 'intersect_ids': True, 'alpha_diversity': M01      65.000000
M03      21.000000
M06      57.000000
M09      68.000000
M12      51.000000
M19      59.000000
M05      33.000000
M07       2.000000
M102    149.333333
M107    117.000000
M112    127.750000
M120    190.500000
M121    246.800000
M125    212.000000
M126    251.076923
M133    225.428571
M134    183.428571
M141    165.333333
M144     94.000000
M148    113.000000
M150    296.500000
M151    131.000000
M155     94.000000
M156    149.857143
M33     127.000000
M34     207.000000
M35     134.000000
M38     145.000000
M39     235.000000
M44     278.000000
M46     129.000000
M47      47.000000
M52     101.500000
M55     343.333333
M64     278.000000
M72     162.000000


DONE: Calculating alpha correlation: chao1 spearman
{'metadata': Metadata
--------
44 IDs x 3 columns
idade:           ColumnProperties(type='numeric')
idade-menarca:   ColumnProperties(type='numeric')
tempo-menopausa: ColumnProperties(type='numeric')

Call to_dataframe() for a tabular representation., 'method': 'pearson', 'intersect_ids': True, 'alpha_diversity': M01      65.000000
M03      21.000000
M06      57.000000
M09      68.000000
M12      51.000000
M19      59.000000
M05      33.000000
M07       2.000000
M102    149.333333
M107    117.000000
M112    127.750000
M120    190.500000
M121    246.800000
M125    212.000000
M126    251.076923
M133    225.428571
M134    183.428571
M141    165.333333
M144     94.000000
M148    113.000000
M150    296.500000
M151    131.000000
M155     94.000000
M156    149.857143
M33     127.000000
M34     207.000000
M35     134.000000
M38     145.000000
M39     235.000000
M44     278.000000
M46     129.000000
M47      47.000000
M52     101.500000
M55   

DONE: Calculating alpha correlation: chao1 pearson
ERROR: Calculating alpha correlation: chao1_ci spearman
ERROR: Calculating alpha correlation: chao1_ci pearson
{'metadata': Metadata
--------
44 IDs x 3 columns
idade:           ColumnProperties(type='numeric')
idade-menarca:   ColumnProperties(type='numeric')
tempo-menopausa: ColumnProperties(type='numeric')

Call to_dataframe() for a tabular representation., 'method': 'spearman', 'intersect_ids': True, 'alpha_diversity': M01      65
M03      21
M06      57
M09      68
M12      51
M19      59
M05      30
M07       2
M102    146
M107    117
M112    105
M120    180
M121    209
M125    197
M126    222
M133    216
M134    177
M141    135
M144     91
M148    113
M150    202
M151    122
M155     91
M156    149
M33     121
M34     171
M35     131
M38     136
M39     232
M44     278
M46     129
M47      47
M52      99
M55     298
M64     275
M72     159
M73     169
M77     112
M78     195
M79     140
M88     210
M93     238
M95     195
M99   

DONE: Calculating alpha correlation: observed_features spearman
{'metadata': Metadata
--------
44 IDs x 3 columns
idade:           ColumnProperties(type='numeric')
idade-menarca:   ColumnProperties(type='numeric')
tempo-menopausa: ColumnProperties(type='numeric')

Call to_dataframe() for a tabular representation., 'method': 'pearson', 'intersect_ids': True, 'alpha_diversity': M01      65
M03      21
M06      57
M09      68
M12      51
M19      59
M05      30
M07       2
M102    146
M107    117
M112    105
M120    180
M121    209
M125    197
M126    222
M133    216
M134    177
M141    135
M144     91
M148    113
M150    202
M151    122
M155     91
M156    149
M33     121
M34     171
M35     131
M38     136
M39     232
M44     278
M46     129
M47      47
M52      99
M55     298
M64     275
M72     159
M73     169
M77     112
M78     195
M79     140
M88     210
M93     238
M95     195
M99     171
Name: observed_features, dtype: int64}
DONE: Calculating alpha correlation: observed_features

DONE: Calculating alpha correlation: shannon spearman
{'metadata': Metadata
--------
44 IDs x 3 columns
idade:           ColumnProperties(type='numeric')
idade-menarca:   ColumnProperties(type='numeric')
tempo-menopausa: ColumnProperties(type='numeric')

Call to_dataframe() for a tabular representation., 'method': 'pearson', 'intersect_ids': True, 'alpha_diversity': M01     4.685409
M03     2.473775
M06     4.084876
M09     4.659494
M12     4.012142
M19     4.516316
M05     3.849078
M07     0.307123
M102    6.169883
M107    5.486286
M112    5.097147
M120    5.996712
M121    5.815968
M125    5.750357
M126    6.118315
M133    6.360023
M134    5.855780
M141    5.434783
M144    5.013189
M148    5.702705
M150    6.273723
M151    5.204093
M155    5.001136
M156    4.904638
M33     5.454111
M34     6.126049
M35     5.428306
M38     5.010866
M39     5.620496
M44     6.331683
M46     4.690242
M47     3.510395
M52     5.572068
M55     6.165377
M64     5.646392
M72     5.963578
M73     5.634037
M7

DONE: Calculating alpha correlation: simpson spearman
{'metadata': Metadata
--------
44 IDs x 3 columns
idade:           ColumnProperties(type='numeric')
idade-menarca:   ColumnProperties(type='numeric')
tempo-menopausa: ColumnProperties(type='numeric')

Call to_dataframe() for a tabular representation., 'method': 'pearson', 'intersect_ids': True, 'alpha_diversity': M01     0.925125
M03     0.748499
M06     0.875417
M09     0.921941
M12     0.885068
M19     0.923832
M05     0.892842
M07     0.103887
M102    0.977082
M107    0.962061
M112    0.950635
M120    0.972714
M121    0.965380
M125    0.959636
M126    0.971463
M133    0.978705
M134    0.968169
M141    0.952579
M144    0.943645
M148    0.971026
M150    0.975615
M151    0.950759
M155    0.930360
M156    0.935778
M33     0.951855
M34     0.972638
M35     0.956623
M38     0.934001
M39     0.950892
M44     0.973343
M46     0.913853
M47     0.861008
M52     0.966034
M55     0.973769
M64     0.954307
M72     0.969125
M73     0.940836
M7

DONE: Calculating alpha correlation: simpson_e spearman
{'metadata': Metadata
--------
44 IDs x 3 columns
idade:           ColumnProperties(type='numeric')
idade-menarca:   ColumnProperties(type='numeric')
tempo-menopausa: ColumnProperties(type='numeric')

Call to_dataframe() for a tabular representation., 'method': 'pearson', 'intersect_ids': True, 'alpha_diversity': M01     0.205471
M03     0.189339
M06     0.140821
M09     0.188394
M12     0.170605
M19     0.222523
M05     0.311067
M07     0.557965
M102    0.298857
M107    0.225280
M112    0.192927
M120    0.203607
M121    0.138206
M125    0.125759
M126    0.157848
M133    0.217402
M134    0.177493
M141    0.156207
M144    0.194997
M148    0.305432
M150    0.203016
M151    0.166461
M155    0.157797
M156    0.104503
M33     0.171656
M34     0.213723
M35     0.175984
M38     0.111409
M39     0.087773
M44     0.134942
M46     0.089985
M47     0.153078
M52     0.297387
M55     0.127931
M64     0.079582
M72     0.203705
M73     0.100014


## Alpha diversity comparisons

Visually and statistically compare groups of alpha diversity values.

[diversity alpha_group_significance](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-group-significance/)

In [16]:
for metric, alpha_values in alpha_diversities.items():
    print(f"Processing alpha_group_significance: {metric}")
    try:
        significance_view = alpha_group_significance(alpha_diversity=alpha_values, metadata=metadata_qa).visualization
        view_path = os.path.join(alpha_path, f'alpha-group-significance-{metric}.qzv')
        significance_view.save(view_path)
        significance_view
        print(f"DONE: Calculating alpha group significance: {metric}")
    except Exception as e:
        print(f"ERROR: Calculating alpha group significance: {metric}")

Processing alpha_group_significance: chao1
{'metadata': Metadata
--------
44 IDs x 8 columns
class-straw:        ColumnProperties(type='categorical')
local-coleta:       ColumnProperties(type='categorical')
idade:              ColumnProperties(type='numeric')
idade-menarca:      ColumnProperties(type='numeric')
tempo-menopausa:    ColumnProperties(type='numeric')
above_10:           ColumnProperties(type='categorical')
menopausa-age-qcut: ColumnProperties(type='categorical')
menopausa-age-bins: ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'alpha_diversity': M01      65.000000
M03      21.000000
M06      57.000000
M09      68.000000
M12      51.000000
M19      59.000000
M05      33.000000
M07       2.000000
M102    149.333333
M107    117.000000
M112    127.750000
M120    190.500000
M121    246.800000
M125    212.000000
M126    251.076923
M133    225.428571
M134    183.428571
M141    165.333333
M144     94.000000
M148    113.000000
M150    296.

DONE: Calculating alpha group significance: chao1
Processing alpha_group_significance: chao1_ci
ERROR: Calculating alpha group significance: chao1_ci
Processing alpha_group_significance: observed_features
{'metadata': Metadata
--------
44 IDs x 8 columns
class-straw:        ColumnProperties(type='categorical')
local-coleta:       ColumnProperties(type='categorical')
idade:              ColumnProperties(type='numeric')
idade-menarca:      ColumnProperties(type='numeric')
tempo-menopausa:    ColumnProperties(type='numeric')
above_10:           ColumnProperties(type='categorical')
menopausa-age-qcut: ColumnProperties(type='categorical')
menopausa-age-bins: ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'alpha_diversity': M01      65
M03      21
M06      57
M09      68
M12      51
M19      59
M05      30
M07       2
M102    146
M107    117
M112    105
M120    180
M121    209
M125    197
M126    222
M133    216
M134    177
M141    135
M144     91
M1

DONE: Calculating alpha group significance: observed_features
Processing alpha_group_significance: shannon
{'metadata': Metadata
--------
44 IDs x 8 columns
class-straw:        ColumnProperties(type='categorical')
local-coleta:       ColumnProperties(type='categorical')
idade:              ColumnProperties(type='numeric')
idade-menarca:      ColumnProperties(type='numeric')
tempo-menopausa:    ColumnProperties(type='numeric')
above_10:           ColumnProperties(type='categorical')
menopausa-age-qcut: ColumnProperties(type='categorical')
menopausa-age-bins: ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'alpha_diversity': M01     4.685409
M03     2.473775
M06     4.084876
M09     4.659494
M12     4.012142
M19     4.516316
M05     3.849078
M07     0.307123
M102    6.169883
M107    5.486286
M112    5.097147
M120    5.996712
M121    5.815968
M125    5.750357
M126    6.118315
M133    6.360023
M134    5.855780
M141    5.434783
M144    5.013189
M148 

DONE: Calculating alpha group significance: shannon
Processing alpha_group_significance: simpson
{'metadata': Metadata
--------
44 IDs x 8 columns
class-straw:        ColumnProperties(type='categorical')
local-coleta:       ColumnProperties(type='categorical')
idade:              ColumnProperties(type='numeric')
idade-menarca:      ColumnProperties(type='numeric')
tempo-menopausa:    ColumnProperties(type='numeric')
above_10:           ColumnProperties(type='categorical')
menopausa-age-qcut: ColumnProperties(type='categorical')
menopausa-age-bins: ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'alpha_diversity': M01     0.925125
M03     0.748499
M06     0.875417
M09     0.921941
M12     0.885068
M19     0.923832
M05     0.892842
M07     0.103887
M102    0.977082
M107    0.962061
M112    0.950635
M120    0.972714
M121    0.965380
M125    0.959636
M126    0.971463
M133    0.978705
M134    0.968169
M141    0.952579
M144    0.943645
M148    0.97102

DONE: Calculating alpha group significance: simpson
Processing alpha_group_significance: simpson_e
{'metadata': Metadata
--------
44 IDs x 8 columns
class-straw:        ColumnProperties(type='categorical')
local-coleta:       ColumnProperties(type='categorical')
idade:              ColumnProperties(type='numeric')
idade-menarca:      ColumnProperties(type='numeric')
tempo-menopausa:    ColumnProperties(type='numeric')
above_10:           ColumnProperties(type='categorical')
menopausa-age-qcut: ColumnProperties(type='categorical')
menopausa-age-bins: ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'alpha_diversity': M01     0.205471
M03     0.189339
M06     0.140821
M09     0.188394
M12     0.170605
M19     0.222523
M05     0.311067
M07     0.557965
M102    0.298857
M107    0.225280
M112    0.192927
M120    0.203607
M121    0.138206
M125    0.125759
M126    0.157848
M133    0.217402
M134    0.177493
M141    0.156207
M144    0.194997
M148    0.305

DONE: Calculating alpha group significance: simpson_e


## Beta diversity analysis

#### Reference
- [diversity beta](https://docs.qiime2.org/2022.8/plugins/available/diversity/beta/): Computes a user-specified beta diversity metric for all pairs of samples in a feature table.
- [Beta diversity metrics](http://scikit-bio.org/docs/0.2.0/generated/skbio.diversity.beta.html)

- Metric Choices('aitchison', 'braycurtis', 'canberra', 'canberra_adkins', 'chebyshev', 'cityblock', 'correlation', 'cosine', 'dice', 'euclidean', 'hamming', 'jaccard', 'jensenshannon', 'kulsinski', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule')

In [17]:
metrics = ('aitchison', 'braycurtis', 'canberra', 'canberra_adkins', 'chebyshev', 'cityblock', 'correlation', 'cosine', 'dice', 'euclidean', 'hamming', 'jaccard', 'jensenshannon', 'kulsinski', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule')
metrics = ('euclidean', 'dice', 'braycurtis', 'correlation', 'cosine', 'matching', 'jaccard')
beta_diversities = dict()
for metric in metrics:
    print(f"Calculating beta diversity: {metric}")
    try:
        beta_diversity = beta(table=tabs, metric=metric, n_jobs=6, pseudocount=1).distance_matrix
        beta_diversities[metric] = beta_diversity
        # Save SampleData[BetaDiversity] Artifact
        file_path = os.path.join(beta_path, f'beta-values-{metric}.qza')
        beta_diversity.save(file_path)
        print(f"DONE: Calculating beta diversity: {metric}")
    except Exception as e:
        print(f"ERROR: Calculating beta diversity: {metric}")

Calculating beta diversity: euclidean
{'metric': 'euclidean', 'pseudocount': 1, 'n_jobs': 6, 'table': <artifact: FeatureTable[Frequency] uuid: b8c70aab-e476-498b-9b85-38d174c191e2>}
{'metric': 'euclidean', 'pseudocount': 1, 'n_jobs': 6, 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
{'metric': 'euclidean', 'pseudocount': 1, 'n_jobs': 6, 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
DONE: Calculating beta diversity: euclidean
Calculating beta diversity: dice
{'metric': 'dice', 'pseudocount': 1, 'n_jobs': 6, 'table': <artifact: FeatureTable[Frequency] uuid: b8c70aab-e476-498b-9b85-38d174c191e2>}
{'metric': 'dice', 'pseudocount': 1, 'n_jobs': 6, 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
{'metric': 'dice', 'pseudocount': 1, 'n_jobs': 6, 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}


/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1761: DataConversionWarning: Data was converted to boolean for metric dice
  warnings.warn(msg, DataConversionWarning)


DONE: Calculating beta diversity: dice
Calculating beta diversity: braycurtis
{'metric': 'braycurtis', 'pseudocount': 1, 'n_jobs': 6, 'table': <artifact: FeatureTable[Frequency] uuid: b8c70aab-e476-498b-9b85-38d174c191e2>}
{'n_jobs': 6, 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
{'n_jobs': 6, 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
DONE: Calculating beta diversity: braycurtis
Calculating beta diversity: correlation
{'metric': 'correlation', 'pseudocount': 1, 'n_jobs': 6, 'table': <artifact: FeatureTable[Frequency] uuid: b8c70aab-e476-498b-9b85-38d174c191e2>}
{'metric': 'correlation', 'pseudocount': 1, 'n_jobs': 6, 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
{'metric': 'correlation', 'pseudocount': 1, 'n_jobs': 6, 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
ERROR: Calculating beta diversity: correlation
Calculating beta dive

DONE: Calculating beta diversity: matching
Calculating beta diversity: jaccard
{'metric': 'jaccard', 'pseudocount': 1, 'n_jobs': 6, 'table': <artifact: FeatureTable[Frequency] uuid: b8c70aab-e476-498b-9b85-38d174c191e2>}
{'n_jobs': 6, 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
{'n_jobs': 6, 'table': 2403 x 44 <class 'biom.table.Table'> with 6331 nonzero entries (5% dense)}
DONE: Calculating beta diversity: jaccard


/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1761: DataConversionWarning: Data was converted to boolean for metric matching
  warnings.warn(msg, DataConversionWarning)
/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1761: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


### Beta group significance

- [diversity beta_group_significance](https://docs.qiime2.org/2022.8/plugins/available/diversity/beta-group-significance/): Determine whether groups of samples are significantly different from one another using a permutation-based statistical test.
- Marti J Anderson. A new method for non-parametric multivariate analysis of variance. Austral ecology, 26(1):32–46, 2001. doi:https://doi.org/10.1111/j.1442-9993.2001.01070.pp.x.

In [18]:
methods = ('permanova', 'anosim', 'permdisp')
for method in methods:
    for metric, beta_diversity in beta_diversities.items():
        print(f'Calculating beta group significance with method {method} and metric {metric}')
        try:
            beta_view = beta_group_significance(distance_matrix=beta_diversity, 
                                                metadata=metadata_qa.get_column(class_col), 
                                                pairwise=True, method=method).visualization
            view_name = os.path.join(beta_path, f'beta-group-significance-{metric}-{method}.qzv')
            beta_view.save(view_name)
            print(f"DONE: Calculating beta group significance: {method} {metric}")
        except Exception as e:
            print(f"ERROR: Calculating beta group significance: {method} {metric}")

Calculating beta group significance with method permanova and metric euclidean
{'metadata': <CategoricalMetadataColumn name='above_10' id_count=44>, 'method': 'permanova', 'pairwise': True, 'permutations': 999, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f633778b1c0>}


DONE: Calculating beta group significance: permanova euclidean
Calculating beta group significance with method permanova and metric dice
{'metadata': <CategoricalMetadataColumn name='above_10' id_count=44>, 'method': 'permanova', 'pairwise': True, 'permutations': 999, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f632446d790>}


DONE: Calculating beta group significance: permanova dice
Calculating beta group significance with method permanova and metric braycurtis
{'metadata': <CategoricalMetadataColumn name='above_10' id_count=44>, 'method': 'permanova', 'pairwise': True, 'permutations': 999, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f63244147c0>}


DONE: Calculating beta group significance: permanova braycurtis
Calculating beta group significance with method permanova and metric matching
{'metadata': <CategoricalMetadataColumn name='above_10' id_count=44>, 'method': 'permanova', 'pairwise': True, 'permutations': 999, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f63244e3fa0>}


DONE: Calculating beta group significance: permanova matching
Calculating beta group significance with method permanova and metric jaccard
{'metadata': <CategoricalMetadataColumn name='above_10' id_count=44>, 'method': 'permanova', 'pairwise': True, 'permutations': 999, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f6324361af0>}


...

/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.002998760695443802 and the largest is 0.024913454301696294.
  warn(


/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.0029987606954438007 and the largest is 0.024913454301696298.
  warn(


DONE: Calculating beta group significance: permdisp matching
Calculating beta group significance with method permdisp and metric jaccard
{'metadata': <CategoricalMetadataColumn name='above_10' id_count=44>, 'method': 'permdisp', 'pairwise': True, 'permutations': 999, 'distance_matrix': <skbio.stats.distance._base.DistanceMatrix object at 0x7f6324290280>}


DONE: Calculating beta group significance: permdisp jaccard


<Figure size 432x288 with 0 Axes>

### Beta group Rarefaction

- [diversity beta_rarefaction](https://docs.qiime2.org/2022.8/plugins/available/diversity/beta-rarefaction/): Repeatedly rarefy a feature table to compare beta diversity results within a given rarefaction depth.  For a given beta diversity metric, this visualizer will provide: an Emperor jackknifed PCoA plot, samples clustered by UPGMA or neighbor joining with support calculation, and a heatmap showing the correlation between rarefaction trials of that beta diversity metric.